In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from miditime.miditime import MIDITime
import datetime
import julian as jl
from midiutil.MidiFile3 import MIDIFile

In [3]:
'''
Combines the Julian date with the TOD that the event occurred and outputs a datetime
'''
def fix_datetime(jul, hour, minute, second):
    #validate input
    if (int(hour) < 0 or int(hour) > 24):
        print("Hour not between 0 and 24 inclusive")
        return
    if (int(minute) < 0 or int(minute) > 60):
        print("Minute not between 0 and 60 inclusive")
        return
    if (second < 0 or second > 60):
        print("Second not between 0 and 60 inclusive")
        return
    #Get Decimal for time of day.
    dec_time = ((int(hour)*60*60) + (int(minute)*60) + second) / 86400
    jul_str = str(jul)
    flt_jul = float(jul_str) + dec_time
    dt = jl.from_jd(round(flt_jul,2), fmt='jd')
    return dt.replace(microsecond=0)

In [4]:
time = '2150'

In [5]:
test_date = fix_datetime(2449579.5, time[0:2], time[2:], 00)
print (test_date)

1994-08-15 21:50:24


I'm going to be doing some data sonification/visualization on wildfire data from 1992-2015 from the wildfire dataset found on Kaggle https://www.kaggle.com/rtatman/188-million-us-wildfires

https://www.nwcg.gov/

Data Sonification Inspiration:
<li> Outside Online episode (need to get reference)\
<li> https://www.revealnews.org/blog/turn-your-data-into-sound-using-our-new-miditime-library/

In [6]:
cnx = sqlite3.connect('/Users/JG/Documents/DataScience/WildFire/FPA_FOD_20170508.sqlite')

In [7]:
#All I need is the DISCOVERY_DATE, and FIRE_SIZE.
df_sonify = pd.read_sql_query("select DISCOVERY_DATE, DISCOVERY_TIME, FIRE_SIZE, LATITUDE, LONGITUDE FROM 'Fires' where STATE = 'MT' and FIRE_YEAR >= 2013 order by DISCOVERY_DATE asc", cnx)
df_sonify.dropna()
df_sonify.head()

,DISCOVERY_DATE,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE
0,2456300.5,1135,2756.0,45.380000,-107.860400
1,2456309.5,1607,0.1,45.606600,-107.458400
2,2456316.5,0605,1.0,45.492500,-110.218611
3,2456317.5,0828,0.1,45.478611,-110.198333
4,2456317.5,1800,4.0,45.611400,-106.516400


In [8]:
for index, row in df_sonify.iterrows():
    if (str(row['DISCOVERY_DATE']).lower() == 'none' or str(row['DISCOVERY_TIME']).lower() == 'none'):
        df_sonify.drop(index, inplace=True)
    else:
        df_sonify.loc[index, 'DISCOVERY_DATE'] = fix_datetime(row["DISCOVERY_DATE"], row["DISCOVERY_TIME"][0:2], row["DISCOVERY_TIME"][2:], 00)

In [9]:
df_sonify = df_sonify.drop(columns='DISCOVERY_TIME')

In [10]:
df_sonify.head()

,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE
0,2013-01-08 11:31:11,2756.0,45.380000,-107.860400
1,2013-01-17 16:04:47,0.1,45.606600,-107.458400
2,2013-01-24 06:00:00,1.0,45.492500,-110.218611
3,2013-01-25 08:24:00,0.1,45.478611,-110.198333
4,2013-01-25 18:00:00,4.0,45.611400,-106.516400


In [11]:
#Get min and max values of FIRE_SIZE for logarithmic scale
c = cnx.cursor()
c.execute("select MAX(FIRE_SIZE) FROM 'Fires' where STATE = 'MT' and FIRE_YEAR >= 2005")
max_fire = c.fetchone()[0]
c.execute("Select MIN(FIRE_SIZE) FROM 'Fires' where STATE = 'MT'and FIRE_YEAR >= 2005")
min_fire = c.fetchone()[0]

In [12]:
print(max_fire)
print(min_fire)

249562.0
0.01


In [13]:
# https://github.com/cirlabs/miditime
mymidi = MIDITime(120, 'myfile.mid', 30, 1, 7, df_sonify.iloc[0]['DISCOVERY_DATE'])

In [14]:
def mag_to_pitch_tuned(magnitude, max_fire, min_fire):
    #Logarithmic scale, reverse order
    scale_pct = mymidi.log_scale_pct(min_fire, max_fire, magnitude, True,direction='log')

    # Pick a range of notes. This allows you to play in a key.
    c_major = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

    #Find the note that matches your data point
    note = mymidi.scale_to_note_classic(scale_pct, c_major)

    #Translate that note to a MIDI pitch
    midi_pitch = mymidi.note_to_midi_pitch(note)

    return midi_pitch

In [15]:
print(mag_to_pitch_tuned(100, max_fire, min_fire))

D4
50


In [16]:
for index, row in df_sonify.iterrows():
    df_sonify.loc[index, 'DAYS_SINCE_EPOCH'] = mymidi.days_since_epoch(row['DISCOVERY_DATE'])

In [17]:
df_sonify.head()

,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,DAYS_SINCE_EPOCH
0,2013-01-08 11:31:11,2756.0,45.380000,-107.860400,0.000000
1,2013-01-17 16:04:47,0.1,45.606600,-107.458400,9.190000
2,2013-01-24 06:00:00,1.0,45.492500,-110.218611,15.770012
3,2013-01-25 08:24:00,0.1,45.478611,-110.198333,16.870012
4,2013-01-25 18:00:00,4.0,45.611400,-106.516400,17.270012


In [18]:
for index, row in df_sonify.iterrows():
    df_sonify.loc[index, 'BEAT'] = mymidi.beat(row['DAYS_SINCE_EPOCH'])

In [19]:
df_sonify.head()

,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,DAYS_SINCE_EPOCH,BEAT
0,2013-01-08 11:31:11,2756.0,45.380000,-107.860400,0.000000,0.00
1,2013-01-17 16:04:47,0.1,45.606600,-107.458400,9.190000,1.51
2,2013-01-24 06:00:00,1.0,45.492500,-110.218611,15.770012,2.59
3,2013-01-25 08:24:00,0.1,45.478611,-110.198333,16.870012,2.77
4,2013-01-25 18:00:00,4.0,45.611400,-106.516400,17.270012,2.84


In [20]:
for index, row in df_sonify.iterrows():
    df_sonify.loc[index, 'SCALED_FIRE_SIZE'] = mag_to_pitch_tuned(row['FIRE_SIZE'], max_fire, min_fire)

A2
C7
C6
C7
F5
G5
C7
G6
E6
G6
C7
C7
C7
C7
C7
E5
A5
E7
C6
G5
G6
D6
G5
F5
B4
E6
B7
G5
E5
C7
B5
G5
A5
A5
A5
A5
A5
B5
C6
A5
G6
F5
C7
B5
C6
D6
A5
A5
C6
E5
G6
F5
G6
B5
A6
D5
G4
B5
C7
A5
A6
F5
D6
G6
C6
G5
G5
C7
C5
C6
B5
E5
C7
C7
E5
C7
D6
E6
C7
C7
C7
A6
C7
C7
C6
C6
A5
E6
C7
G6
G6
C7
A5
C7
B4
C7
C7
F4
F5
C7
D5
A6
C6
F6
C6
B6
G6
C7
C6
C6
C7
B5
A5
C7
A6
F5
E7
C7
G6
E6
A6
C7
B5
E6
A6
C7
D5
A5
F5
F5
E6
C7
C7
E6
D6
C7
C7
C7
A6
G6
G6
E6
E6
E6
F5
E6
C7
C7
C7
C7
G6
C7
A5
C7
A5
C6
E5
B5
F4
G6
G6
G5
G6
G5
A6
F5
B7
B7
C7
C7
B7
G4
C7
C7
C7
C7
G6
A5
G6
E6
A3
F4
E5
B4
A4
C6
G3
C4
F5
B5
A6
F4
D5
C7
C6
A6
C7
C7
C6
C7
E6
G6
C7
C7
C7
F6
C7
C7
C7
E6
C7
C7
C7
C7
C7
C7
G5
G6
B7
B7
A6
C7
C7
C7
E6
C6
E6
C7
D5
C7
C7
C7
C7
C7
C7
C7
E6
C7
C7
B7
B7
F6
E6
E7
F6
B5
G5
D6
C7
C7
C7
C7
C6
A5
C7
C7
B7
C7
C7
C7
C7
C6
C7
C6
A5
C7
G6
C6
F5
C5
C7
C7
C7
C7
C7
C7
C7
C6
C7
C7
C7
C7
C7
D6
C7
B5
C7
C7
C7
C7
C7
C6
C7
A6
A6
G5
A6
C7
C7
C7
G6
G6
C7
C7
F5
B5
C7
C7
F6
E6
C7
C7
C7
E5
C7
C7
C7
C7
C7
C7
C7
C7
C7
F6
C7
C6
D6
C7
C7
E4
C7
C7
C7
C

In [21]:
df_sonify.head()

,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,DAYS_SINCE_EPOCH,BEAT,SCALED_FIRE_SIZE
0,2013-01-08 11:31:11,2756.0,45.380000,-107.860400,0.000000,0.00,33.0
1,2013-01-17 16:04:47,0.1,45.606600,-107.458400,9.190000,1.51,84.0
2,2013-01-24 06:00:00,1.0,45.492500,-110.218611,15.770012,2.59,72.0
3,2013-01-25 08:24:00,0.1,45.478611,-110.198333,16.870012,2.77,84.0
4,2013-01-25 18:00:00,4.0,45.611400,-106.516400,17.270012,2.84,65.0


In [22]:
df_mapping = df_sonify

In [23]:
df_mapping.head()

,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,DAYS_SINCE_EPOCH,BEAT,SCALED_FIRE_SIZE
0,2013-01-08 11:31:11,2756.0,45.380000,-107.860400,0.000000,0.00,33.0
1,2013-01-17 16:04:47,0.1,45.606600,-107.458400,9.190000,1.51,84.0
2,2013-01-24 06:00:00,1.0,45.492500,-110.218611,15.770012,2.59,72.0
3,2013-01-25 08:24:00,0.1,45.478611,-110.198333,16.870012,2.77,84.0
4,2013-01-25 18:00:00,4.0,45.611400,-106.516400,17.270012,2.84,65.0


In [24]:
start_time = df_sonify['BEAT'][0]

In [25]:
note_list = []

for index, row in df_sonify.iterrows():
    note_list.append([
        row['BEAT'] - start_time,
        int(row['SCALED_FIRE_SIZE']),
        100,  # velocity
        1  # duration, in beats
    ])

In [ ]:
print(note_list)

In [26]:
track    = 0
channel  = 0
time     = 0    # In beats
tempo    = mymidi.tempo   # In BPM

print (tempo)

120


In [27]:
MyMIDI = MIDIFile(1)  # One track
MyMIDI.addTempo(track, time, tempo)

In [28]:
for n in note_list:
    MyMIDI.addNote(track, channel, n[1], n[0], n[3], n[2])

In [29]:
with open("fire_sound_test_multi.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

In [30]:
pygame.init()
pygame.mixer.music.load("fire_sound_test_multi.mid")
pygame.mixer.music.play()

In [31]:
pygame.mixer.music.stop()

In [32]:
pygame.quit()

In [33]:
df_mapping.head()

,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,DAYS_SINCE_EPOCH,BEAT,SCALED_FIRE_SIZE
0,2013-01-08 11:31:11,2756.0,45.380000,-107.860400,0.000000,0.00,33.0
1,2013-01-17 16:04:47,0.1,45.606600,-107.458400,9.190000,1.51,84.0
2,2013-01-24 06:00:00,1.0,45.492500,-110.218611,15.770012,2.59,72.0
3,2013-01-25 08:24:00,0.1,45.478611,-110.198333,16.870012,2.77,84.0
4,2013-01-25 18:00:00,4.0,45.611400,-106.516400,17.270012,2.84,65.0


In [34]:
df_mapping['BEAT']

0         0.00
1         1.51
2         2.59
3         2.77
4         2.84
5         3.01
6         4.29
7         4.29
8         4.80
9         5.20
10        5.96
11        6.30
12        6.43
13        6.46
14        8.72
15        8.72
16        9.89
17        9.87
18        9.87
19       10.07
20       10.23
21       10.22
22       10.38
23       10.73
24       10.79
25       11.53
26       11.99
27       12.34
28       12.33
29       12.54
         ...  
4398    168.00
4399    168.05
4400    168.13
4401    168.09
4402    168.24
4403    168.27
4404    168.28
4405    168.39
4406    168.92
4407    168.82
4408    169.39
4409    169.56
4410    170.75
4411    170.71
4412    170.85
4413    170.90
4414    170.93
4415    170.98
4416    171.44
4417    171.70
4418    173.18
4419    173.63
4420    173.63
4421    174.80
4422    174.95
4423    174.97
4424    174.95
4425    174.97
4426    175.04
4427    175.82
Name: BEAT, Length: 4427, dtype: float64

In [35]:
len(df_mapping.index)

4427

In [36]:
import math

In [37]:
df_mapping.head()

,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,DAYS_SINCE_EPOCH,BEAT,SCALED_FIRE_SIZE
0,2013-01-08 11:31:11,2756.0,45.380000,-107.860400,0.000000,0.00,33.0
1,2013-01-17 16:04:47,0.1,45.606600,-107.458400,9.190000,1.51,84.0
2,2013-01-24 06:00:00,1.0,45.492500,-110.218611,15.770012,2.59,72.0
3,2013-01-25 08:24:00,0.1,45.478611,-110.198333,16.870012,2.77,84.0
4,2013-01-25 18:00:00,4.0,45.611400,-106.516400,17.270012,2.84,65.0


In [38]:
#Reset dataframe index for animation. This is because we deleted some records earlier.
df_mapping = df_mapping.reset_index(drop=True)

In [39]:
from mido import MidiFile

mid = MidiFile('wildfire_sonification.mid')
mid.length
#for i, track in enumerate(mid.tracks):
#    print('Track {}: {}'.format(i, track.name))
#    for msg in track:
#        print(msg)   

109.03020833333623

In [40]:
mid.ticks_per_beat

960

In [41]:
#create a new column with number of ticks:
for index, row in df_mapping.iterrows():
    df_mapping.loc[index, 'TICKS'] = row['BEAT'] * mid.ticks_per_beat

In [42]:
#Determine how many milliseconds per tick: 60000 / (BPM * PPQ)
millis_per_tick = 60000.0 / (120 * mid.ticks_per_beat)
print(millis_per_tick)

0.5208333333333334


In [43]:
#create a new column with time elapsed in milliseconds
for index, row in df_mapping.iterrows():
    df_mapping.loc[index, 'MILLIS'] = row['TICKS'] * millis_per_tick

In [65]:
df_mapping = df_mapping.sort_values(by=['BEAT'])

In [66]:
#Create an index list for frames for the matplotlib animation
frames_list = list(df_mapping['MILLIS'])

In [67]:
#convert from floats to ints:
frames_list = [int(i) for i in frames_list]

In [68]:
frames_list_test = frames_list[:20]

In [69]:
print(frames_list_test)

[0, 755, 1295, 1385, 1420, 1505, 2145, 2145, 2400, 2600, 2980, 3150, 3215, 3230, 4360, 4360, 4935, 4935, 4945, 5035]


In [70]:
frames_list_test_deltas = [x - frames_list_test[i - 1] for i, x in enumerate(frames_list_test)][1:]
print(frames_list_test_deltas)

[755, 540, 90, 35, 85, 640, 0, 255, 200, 380, 170, 65, 15, 1130, 0, 575, 0, 10, 90]


In [ ]:
#create index list for frames, i.e. how many cycles each frame will be displayed
#https://stackoverflow.com/questions/49796314/animate-with-variable-time
for i, item in enumerate(frames_list_test_deltas):
    frames_list_test_deltas.extend([i]*item)
print(frames_list_test_deltas)


#Note: test the first 20 and figure out why it is incrementing so much in animate vs the example.....

In [ ]:
'''
***Each year has a different color plot point, perhaps color is defined in dataframe and is assigned as part of data cleanup.??.??.

***Need to be sure not to get index out of bounds exception, so only increment if index <= last record

Iterate thru midi file with pygame
    Q up next record in dataframe with beat Number:
    while iterating thru midi file 
        if record beat = loop beat generate new image with plot point and increment record index to next record
        if record beat != loop beat repeat existing image with existing plot point.
'''


%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation



map_terrain = cimgt.Stamen('terrain-background')
fig = plt.figure(figsize=(19.2,10.8))
ax = plt.axes(projection=map_terrain.crs)
ax.set_extent([-116.5, -103, 44, 49.5]) #Montana boundary box
ax.add_image(map_terrain,8,zorder=0)
ax.add_feature(cfeature.STATES, zorder=1)

def update(frame_number):
    print(frame_number)
    image = ax.plot(df_mapping['LONGITUDE'][frame_number], df_mapping['LATITUDE'][frame_number], marker='o', color='r', 
             markersize=df_mapping['SCALED_FIRE_SIZE'][frame_number], alpha=0.7, transform=map_terrain.crs.as_geodetic(), zorder=2)
    
    return image,

anim = FuncAnimation(fig, update, interval=1, frames=frames_list)#,save_count=200)#len(df_mapping.index))

In [ ]:
#HTML(anim.to_html5_video())
anim.save('wildfire_mov.mp4', 'ffmpeg', fps=30)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML

fig = plt.figure()

x=[20,23,25,27,29,31]
y=[10,12,14,16,17,19]
t=[2,10,1,4,3,9]

#create index list for frames, i.e. how many cycles each frame will be displayed
frame_t = []
for i, item in enumerate(t):
    frame_t.extend([i] * item)
frame_t

def init():
    fig.clear()

#animation function
def animate(i): 
    print(i)
    #prevent autoscaling of figure
    plt.xlim(15, 35)
    plt.ylim( 5, 25)
    #set new point
    plt.scatter(x[i], y[i], c = "b")

#animate scatter plot
ani = anim.FuncAnimation(fig, animate, init_func = init, 
                         frames = frame_t, interval = 100, repeat = True)
HTML(ani.to_html5_video())